In [1]:
# We import the libraries that are needed
# for solving linear programming problems
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# For performance analysis purposes
from time import process_time

# For saving the results returned by the solver
from pandas import Series, DataFrame

# For better code readability and maintanability
from typing import Final

# Define the constants
NUMBER_OF_INTENSIVISTS: Final = 24
WEEK: Final = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
SHIFTS: Final = ['MiA', 'MiC', 'MnA', 'MnC',
                 'AiA', 'AiC', 'AnA', 'AnC',
                 'NiA', 'NiC', 'NnA', 'NnC']

In [2]:
# We define the continuous model
nonlinear_model = pyo.ConcreteModel()

# We create the labels of the 1st dimension of the continuous model
# that correspondes to the rows of our previously seen matrix
i_labels = SHIFTS

# We check that the the length of the 1st dimension
# is equal to 12
assert len(i_labels) == len(SHIFTS)

# We create the labels of the 2nd dimension of the continuous model
# that corresponds to the columns of our previously seen matrix
j_labels = WEEK

# We check the length of the 2nd dimension
# to be equal to 7
assert len(j_labels) == 7

# We initialize the model dimensions with the labels
# that were defined above
nonlinear_model.i = pyo.Set(initialize = i_labels, doc = "scenarios")
nonlinear_model.j = pyo.Set(initialize = j_labels, doc = "days of the week")

patient_forecast = {

    # In-Python used notation:
    # example: MiA <- morning shift, infectious disease, adult section
    # example: NnC <- night shift, non-infectious disease, children section

    # MORNING INFECTIOUS
    ('MiA', 'Mon'): 15,
    ('MiA', 'Tue'): 12,
    ('MiA', 'Wed'): 18,
    ('MiA', 'Thu'): 13,
    ('MiA', 'Fri'): 10,
    ('MiA', 'Sat'): 11,
    ('MiA', 'Sun'): 11,
    
    ('MiC', 'Mon'): 10,
    ('MiC', 'Tue'): 12,
    ('MiC', 'Wed'): 15,
    ('MiC', 'Thu'): 19,
    ('MiC', 'Fri'): 20,
    ('MiC', 'Sat'): 21,
    ('MiC', 'Sun'): 21,
    
    # MORNING NON-INFECTIOUS
    ('MnA', 'Mon'): 10,
    ('MnA', 'Tue'): 8,
    ('MnA', 'Wed'): 5,
    ('MnA', 'Thu'): 5,
    ('MnA', 'Fri'): 4,
    ('MnA', 'Sat'): 4,
    ('MnA', 'Sun'): 3,

    ('MnC', 'Mon'): 4,
    ('MnC', 'Tue'): 3,
    ('MnC', 'Wed'): 3,
    ('MnC', 'Thu'): 3,
    ('MnC', 'Fri'): 3,
    ('MnC', 'Sat'): 3,
    ('MnC', 'Sun'): 2,

    # AFTERNOON INFECTIOUS
    ('AiA', 'Mon'): 16,
    ('AiA', 'Tue'): 14,
    ('AiA', 'Wed'): 16,
    ('AiA', 'Thu'): 13,
    ('AiA', 'Fri'): 9,
    ('AiA', 'Sat'): 9,
    ('AiA', 'Sun'): 8,

    ('AiC', 'Mon'): 9,
    ('AiC', 'Tue'): 12,
    ('AiC', 'Wed'): 15,
    ('AiC', 'Thu'): 17,
    ('AiC', 'Fri'): 21,
    ('AiC', 'Sat'): 22,
    ('AiC', 'Sun'): 23,
    
    # AFTERNOON NON-INFECTIOUS
    ('AnA', 'Mon'): 11,
    ('AnA', 'Tue'): 9,
    ('AnA', 'Wed'): 4,
    ('AnA', 'Thu'): 5,
    ('AnA', 'Fri'): 3,
    ('AnA', 'Sat'): 2,
    ('AnA', 'Sun'): 2,

    ('AnC', 'Mon'): 4,
    ('AnC', 'Tue'): 3,
    ('AnC', 'Wed'): 3,
    ('AnC', 'Thu'): 3,
    ('AnC', 'Fri'): 2,
    ('AnC', 'Sat'): 2,
    ('AnC', 'Sun'): 2,

    # NIGHT INFECTIOUS
    ('NiA', 'Mon'): 14,
    ('NiA', 'Tue'): 14,
    ('NiA', 'Wed'): 17,
    ('NiA', 'Thu'): 12,
    ('NiA', 'Fri'): 7,
    ('NiA', 'Sat'): 8,
    ('NiA', 'Sun'): 7,
    
    ('NiC', 'Mon'): 10,
    ('NiC', 'Tue'): 11,
    ('NiC', 'Wed'): 13,
    ('NiC', 'Thu'): 18,
    ('NiC', 'Fri'): 22,
    ('NiC', 'Sat'): 23,
    ('NiC', 'Sun'): 24,
    
    # NIGHT NON-INFECTIOUS
    ('NnA', 'Mon'): 12,
    ('NnA', 'Tue'): 10,
    ('NnA', 'Wed'): 5,
    ('NnA', 'Thu'): 4,
    ('NnA', 'Fri'): 2,
    ('NnA', 'Sat'): 2,
    ('NnA', 'Sun'): 2,
    
    ('NnC', 'Mon'): 4,
    ('NnC', 'Tue'): 4,
    ('NnC', 'Wed'): 3,
    ('NnC', 'Thu'): 2,
    ('NnC', 'Fri'): 2,
    ('NnC', 'Sat'): 3,
    ('NnC', 'Sun'): 2
}

# We check the length of matrix
# to be equal to 84 (12 x 7)
assert len(patient_forecast.items()) == len(WEEK)*len(SHIFTS)

# We define the forescated number of patients per scenario and
# days of the week into the model. This information is
# crucial in the solution to the problem
nonlinear_model.patients = pyo.Param(nonlinear_model.i, nonlinear_model.j,
    initialize = patient_forecast, doc = "Forescasted patients per scenario and day of the week")

# Print the basic parts of the continuous model
nonlinear_model.pprint()

3 Set Declarations
    i : scenarios
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {'MiA', 'MiC', 'MnA', 'MnC', 'AiA', 'AiC', 'AnA', 'AnC', 'NiA', 'NiC', 'NnA', 'NnC'}
    j : days of the week
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'}
    patients_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   84 : {('MiA', 'Mon'), ('MiA', 'Tue'), ('MiA', 'Wed'), ('MiA', 'Thu'), ('MiA', 'Fri'), ('MiA', 'Sat'), ('MiA', 'Sun'), ('MiC', 'Mon'), ('MiC', 'Tue'), ('MiC', 'Wed'), ('MiC', 'Thu'), ('MiC', 'Fri'), ('MiC', 'Sat'), ('MiC', 'Sun'), ('MnA', 'Mon'), ('MnA', 'Tue'), ('MnA', 'Wed'), ('MnA', 'Thu'), ('MnA', 'Fri'), ('MnA', 'Sat'), ('MnA', 'Sun'), ('MnC', 'Mon'), ('MnC', 'Tue'), ('MnC', 'Wed'), ('MnC', 'Thu')

In [3]:
# We define variables x(i,j)
# that represent the number of intensivists
# per scenario, per day of the week
nonlinear_model.x = pyo.Var(nonlinear_model.i, nonlinear_model.j,
    doc = "Number of intensivists",
    within = pyo.NonNegativeReals)

print(nonlinear_model.x.display())

x : Number of intensivists
    Size=84, Index=x_index
    Key            : Lower : Value : Upper : Fixed : Stale : Domain
    ('AiA', 'Fri') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Mon') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Sat') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Sun') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Thu') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Tue') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiA', 'Wed') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiC', 'Fri') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiC', 'Mon') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiC', 'Sat') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiC', 'Sun') :     0 :  None :  None : False :  True : NonNegativeReals
    ('AiC', 'Thu') :

In [4]:
# We define the objective function to be used
def nl_objective_rule(nl_model: pyo.ConcreteModel) -> float:
    return sum([((nl_model.x[i,j]/nl_model.patients[i,j])-(2/15))**2 for count, i in enumerate(nl_model.i) for j in nl_model.j])

In [5]:
# We instantiate the objective function for the model which is passed as a rule
# although Pyomo minimizes by default, we still explicitly specify this
nonlinear_model.objective = pyo.Objective(
                                            rule = nl_objective_rule,
                                            sense = pyo.minimize,
                                             doc = "Real-to-Optimal Intensivist-to-Patient ratio"
                                             )

# Visualize the objective function
nonlinear_model.objective.pprint()

objective : Real-to-Optimal Intensivist-to-Patient ratio
    Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : (0.06666666666666667*x[MiA,Mon] - 0.13333333333333333)**2 + (0.08333333333333333*x[MiA,Tue] - 0.13333333333333333)**2 + (0.05555555555555555*x[MiA,Wed] - 0.13333333333333333)**2 + (0.07692307692307693*x[MiA,Thu] - 0.13333333333333333)**2 + (0.1*x[MiA,Fri] - 0.13333333333333333)**2 + (0.09090909090909091*x[MiA,Sat] - 0.13333333333333333)**2 + (0.09090909090909091*x[MiA,Sun] - 0.13333333333333333)**2 + (0.1*x[MiC,Mon] - 0.13333333333333333)**2 + (0.08333333333333333*x[MiC,Tue] - 0.13333333333333333)**2 + (0.06666666666666667*x[MiC,Wed] - 0.13333333333333333)**2 + (0.05263157894736842*x[MiC,Thu] - 0.13333333333333333)**2 + (0.05*x[MiC,Fri] - 0.13333333333333333)**2 + (0.047619047619047616*x[MiC,Sat] - 0.13333333333333333)**2 + (0.047619047619047616*x[MiC,Sun] - 0.13333333333333333)**2 + (0.1*x[MnA,Mon] - 0.13333333333333333)*

In [6]:
# Function that establishes the constraint by which the solution to the number of doctors
# cannot be larger than the number of available medical staff
def nl_MaxStaffPerDay(nl_model: pyo.ConcreteModel, j: str) -> bool:
    return sum(nl_model.x[i,j] for i in nl_model.i) <= NUMBER_OF_INTENSIVISTS

In [7]:
# This constraint is now added to the model
nonlinear_model.maxStaffPerDay = pyo.Constraint(
    nonlinear_model.j,
    rule = nl_MaxStaffPerDay,
    doc = "Maximum number of doctors per day")

# Visualize the constraint maxStaffPerDay
nonlinear_model.maxStaffPerDay.pprint()

maxStaffPerDay : Maximum number of doctors per day
    Size=7, Index=j, Active=True
    Key : Lower : Body                                                                                                                                                      : Upper : Active
    Fri :  -Inf : x[MiA,Fri] + x[MiC,Fri] + x[MnA,Fri] + x[MnC,Fri] + x[AiA,Fri] + x[AiC,Fri] + x[AnA,Fri] + x[AnC,Fri] + x[NiA,Fri] + x[NiC,Fri] + x[NnA,Fri] + x[NnC,Fri] :  24.0 :   True
    Mon :  -Inf : x[MiA,Mon] + x[MiC,Mon] + x[MnA,Mon] + x[MnC,Mon] + x[AiA,Mon] + x[AiC,Mon] + x[AnA,Mon] + x[AnC,Mon] + x[NiA,Mon] + x[NiC,Mon] + x[NnA,Mon] + x[NnC,Mon] :  24.0 :   True
    Sat :  -Inf : x[MiA,Sat] + x[MiC,Sat] + x[MnA,Sat] + x[MnC,Sat] + x[AiA,Sat] + x[AiC,Sat] + x[AnA,Sat] + x[AnC,Sat] + x[NiA,Sat] + x[NiC,Sat] + x[NnA,Sat] + x[NnC,Sat] :  24.0 :   True
    Sun :  -Inf : x[MiA,Sun] + x[MiC,Sun] + x[MnA,Sun] + x[MnC,Sun] + x[AiA,Sun] + x[AiC,Sun] + x[AnA,Sun] + x[AnC,Sun] + x[NiA,Sun] + x[NiC,Sun] + x[NnA,Sun] +

In [8]:
# Function that implements the minimal Intensivist-to-Patients ratio that
# is minimally admissible taking into account healthcare standards
# for Level I (which only requires physicians during weekdays)
def nl_MinProportion(nl_model: pyo.ConcreteModel, constraintList: pyo.ConstraintList) -> None:
    for i in SHIFTS:
        for j in WEEK:
            constraintList.add(expr = nl_model.x[i,j]/nl_model.patients[i,j] >= 1/14)

    assert len(constraintList) == len(SHIFTS)*len(WEEK)
    constraintList.pprint()
    
    return

In [9]:
# Function that implements the minimal Intensivist-to-Patients ratio that
# is minimally admissible taking into account healthcare standards
# for Level I (which only requires physicians during weekdays)
def nl_MaxProportion(nl_model: pyo.ConcreteModel, constraintList: pyo.ConstraintList) -> None:
    for i in SHIFTS:
        for j in WEEK:
            constraintList.add(expr = nl_model.x[i,j]/nl_model.patients[i,j] <= 1/3)

    assert len(constraintList) == len(SHIFTS)*len(WEEK)
    constraintList.pprint()
    
    return

In [10]:
nonlinear_model.minproportions = pyo.ConstraintList()
nl_MinProportion(nonlinear_model, nonlinear_model.minproportions)

minproportions : Size=84, Index=minproportions_index, Active=True
    Key : Lower               : Body                            : Upper : Active
      1 : 0.07142857142857142 :  0.06666666666666667*x[MiA,Mon] :  +Inf :   True
      2 : 0.07142857142857142 :  0.08333333333333333*x[MiA,Tue] :  +Inf :   True
      3 : 0.07142857142857142 :  0.05555555555555555*x[MiA,Wed] :  +Inf :   True
      4 : 0.07142857142857142 :  0.07692307692307693*x[MiA,Thu] :  +Inf :   True
      5 : 0.07142857142857142 :                  0.1*x[MiA,Fri] :  +Inf :   True
      6 : 0.07142857142857142 :  0.09090909090909091*x[MiA,Sat] :  +Inf :   True
      7 : 0.07142857142857142 :  0.09090909090909091*x[MiA,Sun] :  +Inf :   True
      8 : 0.07142857142857142 :                  0.1*x[MiC,Mon] :  +Inf :   True
      9 : 0.07142857142857142 :  0.08333333333333333*x[MiC,Tue] :  +Inf :   True
     10 : 0.07142857142857142 :  0.06666666666666667*x[MiC,Wed] :  +Inf :   True
     11 : 0.07142857142857142 :  0.05263157

In [11]:
nonlinear_model.maxproportions = pyo.ConstraintList()
nl_MaxProportion(nonlinear_model, nonlinear_model.maxproportions)

maxproportions : Size=84, Index=maxproportions_index, Active=True
    Key : Lower : Body                            : Upper              : Active
      1 :  -Inf :  0.06666666666666667*x[MiA,Mon] : 0.3333333333333333 :   True
      2 :  -Inf :  0.08333333333333333*x[MiA,Tue] : 0.3333333333333333 :   True
      3 :  -Inf :  0.05555555555555555*x[MiA,Wed] : 0.3333333333333333 :   True
      4 :  -Inf :  0.07692307692307693*x[MiA,Thu] : 0.3333333333333333 :   True
      5 :  -Inf :                  0.1*x[MiA,Fri] : 0.3333333333333333 :   True
      6 :  -Inf :  0.09090909090909091*x[MiA,Sat] : 0.3333333333333333 :   True
      7 :  -Inf :  0.09090909090909091*x[MiA,Sun] : 0.3333333333333333 :   True
      8 :  -Inf :                  0.1*x[MiC,Mon] : 0.3333333333333333 :   True
      9 :  -Inf :  0.08333333333333333*x[MiC,Tue] : 0.3333333333333333 :   True
     10 :  -Inf :  0.06666666666666667*x[MiC,Wed] : 0.3333333333333333 :   True
     11 :  -Inf :  0.05263157894736842*x[MiC,Thu] : 0.

In [12]:
Solver = SolverFactory('ipopt')
nonlinear_model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
Results = Solver.solve(nonlinear_model, tee=True)
print(Results)

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:      252
Number of nonzeros in Lagrangian Hessian.............:       84

Total number of variables............................:       84
                     variables with only lower bounds:       84
                variables with lower and upper bounds:        0


In [13]:
for i in SHIFTS:
    for j in WEEK:
        print(f"x[{i},{j}] = {pyo.value(nonlinear_model.x[i,j])}")

x[MiA,Mon] = 2.000000396030046
x[MiA,Tue] = 1.6000003245535332
x[MiA,Wed] = 2.400000467434914
x[MiA,Thu] = 1.7333336824732744
x[MiA,Fri] = 1.3333336080636393
x[MiA,Sat] = 1.4666669663502099
x[MiA,Sun] = 1.4666669669665893
x[MiC,Mon] = 1.3333336056185965
x[MiC,Tue] = 1.6000003245535332
x[MiC,Wed] = 2.000000396827746
x[MiC,Thu] = 2.5333338257502964
x[MiC,Fri] = 2.6666671878718913
x[MiC,Sat] = 2.800000540692921
x[MiC,Sun] = 2.800000542889503
x[MnA,Mon] = 1.3333336056185965
x[MnA,Tue] = 1.0666668878788708
x[MnA,Wed] = 0.6666668069606914
x[MnA,Thu] = 0.666666807106516
x[MnA,Fri] = 0.5333334465382278
x[MnA,Sat] = 0.5333334464070717
x[MnA,Sun] = 0.4000000852949354
x[MnC,Mon] = 0.5333334461633263
x[MnC,Tue] = 0.4000000852218513
x[MnC,Wed] = 0.4000000851408456
x[MnC,Thu] = 0.4000000851932676
x[MnC,Fri] = 0.4000000853251875
x[MnC,Sat] = 0.40000008525140257
x[MnC,Sun] = 0.2666667238182922
x[AiA,Mon] = 2.133333753109976
x[AiA,Tue] = 1.8666670410232302
x[AiA,Wed] = 2.1333337540181962
x[AiA,Thu] = 1

In [14]:
print(pyo.value(nonlinear_model.objective))

6.33458445962465e-14


In [15]:
nonlinear_model.dual.display()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                 : Value
    maxStaffPerDay[Fri] : -2.1983065470191738e-10
    maxStaffPerDay[Mon] : -2.5497145161718736e-10
    maxStaffPerDay[Sat] : -2.3192249103556756e-10
    maxStaffPerDay[Sun] : -2.2478344815867113e-10
    maxStaffPerDay[Thu] : -2.4164031566019587e-10
    maxStaffPerDay[Tue] :  -2.357306949095915e-10
    maxStaffPerDay[Wed] : -2.4994684119041334e-10
     maxproportions[10] : -1.1808945866153216e-08
     maxproportions[11] : -1.1825489460564069e-08
     maxproportions[12] : -1.1818722204922236e-08
     maxproportions[13] : -1.1829604324698924e-08
     maxproportions[14] : -1.1825452025156918e-08
     maxproportions[15] : -1.1784435886830801e-08
     maxproportions[16] : -1.1770511027176105e-08
     maxproportions[17] : -1.1757899219081997e-08
     maxproportions[18] : -1.1756707813275521e-08
     maxproportions[19] : -1.1749587771836558e-08
      maxproportions[1] : -1.1811314365989172e-08
     maxpropor